# Validation - BAC response of models

This notebook evaluates qualitatively the capability of fitted models to reproduce BAC behavior.

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import neuroplotlib as nplt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import LFPy
import neuron
import numpy as np

import shutil

import multimodalfitting as mf

# %matplotlib notebook
%matplotlib widget

In [ ]:
base_path = Path("../..")

In [ ]:
colors_dict = {"soma": "C0",
               "all": "C1",
               "sections": "C2",
               "single": "C3"}
figsize = (10, 7)

## Load results and define model

In [ ]:
probe_type = "planar"
model_name = "hay_ais" # "hay", "hay_ais", "hay_ais_hillock"
cell_models_folder = base_path / "cell_models"

model_folder = cell_models_folder / f"{model_name}"

In [ ]:
# change this with folder containing your pkl file
results_date = '220127'  # '211124' '220111' # 
result_folder = base_path / "results" / results_date

In [ ]:
pkl_file_name = "runs.pkl"

data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

In [ ]:
# set strategy column
df_model.loc[:, "strategy"] = df_model["extra_strategy"].values.copy()
df_model.loc[df_model["feature_set"] == "soma", "strategy"] = "soma"

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name)
cell_release = mf.create_ground_truth_model(model_name=model_name, release=True)

# probe = mf.define_electrode(probe_type=probe_type)
probe=None

param_names = [param.name for param in cell.params.values() if not param.frozen]
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

# Test calcium spike

In [ ]:
opt_results = {}
for strategy in np.unique(df_model.strategy):
    opt_results[strategy] = {}
    opt_df = df_model.query(f"strategy == '{strategy}'")
    best_idx = np.argmin(opt_df.best_fitness)
    params_sample = opt_df.iloc[best_idx]
    params_dict = {k: v for k, v in zip(param_names, params_sample.best_params)}
    opt_results[strategy]["best_fitness"] = params_sample.best_fitness
    opt_results[strategy]["best_params"] = params_dict
    print(f"{strategy} --  best fitness: {params_sample.best_fitness}")

In [ ]:
bac_kwargs = {"syn_weight": 2, "pulse_amp": 1}

In [ ]:
print("Computing RELEASE")
responses_release = mf.utils.simulate_BAC_responses(cell, params_release, sim, **bac_kwargs)

for strategy in np.unique(df_model.strategy):
    print(f"Computing {strategy}")
    best_params = opt_results[strategy]["best_params"]

    responses = mf.utils.simulate_BAC_responses(cell, best_params, sim, **bac_kwargs)
    opt_results[strategy]["responses"] = responses

In [ ]:
responses_release

In [ ]:
ax_dict = {}

loc_names = responses_release.keys()
for loc in responses_release:
    fig, ax = plt.subplots(figsize=figsize)
    ax_dict[loc] = ax
    
    resp = responses_release[loc]
    ax.plot(resp["time"], resp["voltage"], alpha=0.8, color="k", label="GT")
    
    for strategy in opt_results:
        responses = opt_results[strategy]["responses"]
        resp = responses[loc]
        ax.plot(resp["time"], resp["voltage"], alpha=0.8, color=colors_dict[strategy], label=strategy.upper())
        
    ax.set_xlabel("time (ms)", fontsize=12)
    ax.set_ylabel("$V_m$ (mV)", fontsize=12)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_title(f"Recording from {loc}", fontsize=15)
#     ax.axvline(ms_before, color="gray", ls="--")
    ax.legend()
